In [1]:
import numpy as np
categories = np.load('./categories.npy')
valid_periods = np.load('./valid_periods.npy')
training_data = np.load('./training_data.npy')

In [2]:
import pandas as pd
training_data_df = pd.DataFrame(training_data)
training_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Columns: 2776 entries, 0 to 2775
dtypes: float64(2776)
memory usage: 1016.6 MB


In [3]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
reshaped_categories = categories.reshape(-1, 1)
onehot_categories = encoder.fit_transform(reshaped_categories)

onehot_categories

/home/ubuntu/hive_retrieval_engine/test_venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [4]:
from sklearn.model_selection import train_test_split

# Split the indices of the time series into training and validation sets
train_indices, val_indices = train_test_split(np.arange(len(training_data_df)), test_size=0.03, random_state=42, stratify=categories)

# Use these indices to create training and validation sets
train_set = training_data_df.iloc[train_indices]
valid_periods_train = valid_periods[train_indices]
onehot_categories_train = onehot_categories[train_indices]

val_set = training_data_df.iloc[val_indices]
valid_periods_val = valid_periods[val_indices]
onehot_categories_val = onehot_categories[val_indices]

train_set.shape, valid_periods_train.shape, onehot_categories_train.shape, val_set.shape, valid_periods_val.shape, onehot_categories_val.shape

((46560, 2776), (46560, 2), (46560, 6), (1440, 2776), (1440, 2), (1440, 6))

In [5]:
def calculate_padding_length(sq, w, t, s):
    cycle = w + t
    if sq <= cycle: return cycle - sq
    else:
        m = (sq - cycle) % s
        if m == 0: return m
        else: return s - m

In [6]:
def create_sequences(df, valid_periods, categories, window, telescope, stride):

    input_sequences = []
    input_categories = []
    output_sequences = []
    

    for i in range(len(valid_periods)):
        start, end = valid_periods[i]
        category = categories[i]
        
        sequence = df.iloc[i, start:end].values
        
        padding_length = calculate_padding_length(len(sequence), window, telescope, stride)
        sequence = np.pad(sequence, (padding_length, 0), mode='constant', constant_values=0)

        for j in range(0, len(sequence) - window - telescope + 1, stride):
            input_seq = sequence[j:(j + window)]
            output_seq = sequence[(j + window):(j + window + telescope)]
            
            input_sequences.append(input_seq)
            input_categories.append(category)
            output_sequences.append(output_seq)

    return np.array(input_sequences), np.array(input_categories), np.array(output_sequences)

In [7]:
window = 100
telescope = 18
stride = 6

In [8]:
X_train, X_train_category, y_train = create_sequences(train_set, valid_periods_train, onehot_categories_train, window, telescope, stride)
X_val, X_val_category, y_val = create_sequences(val_set, valid_periods_val, onehot_categories_val, window, telescope, stride)
X_train.shape, X_train_category.shape, y_train.shape, X_val.shape, X_val_category.shape, y_val.shape

((835049, 100),
 (835049, 6),
 (835049, 18),
 (27142, 100),
 (27142, 6),
 (27142, 18))

In [9]:
import tensorflow as tf

2023-12-21 12:15:17.134680: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-21 12:15:17.178062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 12:15:17.946400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
# https://pylessons.com/transformers-introduction
def positional_encoding(length: int, depth: int):
    """
    Generates a positional encoding for a given length and depth.

    Args:
        length (int): The length of the input sequence.
        depth (int): The depth that represents the dimensionality of the encoding.

    Returns:
        tf.Tensor: The positional encoding of shape (length, depth).
    """
    depth = depth / 2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)], axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

In [11]:
class PositionalEncoding(tf.keras.layers.Layer):
    """
    A positional encoding layer with a positional encoding that helps the Transformer
    to understand the relative position of the inputs. 

    Methods:
        call: Performs the forward pass of the layer.
    """
    
    def __init__(self, max_length: int, d_model: int):
        """
        Args:
            max_length (int): The size of the maximum input sequence.
            d_model (int): The dimensionality of the pos encoding.
        """
        super().__init__()
        self.max_length = max_length
        self.d_model = d_model
        self.pos_encoding = positional_encoding(length=max_length, depth=d_model)
        
    def call(self, x: tf.Tensor) -> tf.Tensor:
        """ Performs the forward pass of the layer.
        
        Args:
            x (tf.Tensor): The input tensor of shape (batch_size, seq_length).

        Returns:
            tf.Tensor: The output sequence with added positional information. The shape is
                (batch_size, seq_length, d_model).
        """
        length = tf.shape(x)[1]
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[tf.newaxis, :length, :]
        return x

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, MultiHeadAttention, LayerNormalization, Dropout, Add

class TimeSeriesSelfAttention(Layer):
    """
    Implements a self-attention mechanism specifically for time series data. The attention mechanism allows each point in 
    the sequence to attend to all other points in the same sequence, capturing the global dependencies within the time series.

    Attributes:
        mha (MultiHeadAttention): The MultiHeadAttention layer.
        layernorm (LayerNormalization): The LayerNormalization layer.
        dropout (Dropout): The Dropout layer to prevent overfitting.
        add (Add): The Add layer to combine inputs and attention outputs.
    """
    def __init__(self, d_model, num_heads, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.layernorm = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(dropout_rate)
        self.add = Add()

    def call(self, inputs, training=False):
        # Self-attention
        attn_output = self.mha(inputs, inputs, inputs)
        attn_output = self.dropout(attn_output, training=training)
        # Add & Norm
        out = self.layernorm(self.add([inputs, attn_output]))
        return out

In [13]:
class FeedForward(Layer):
    """
    A class that implements the feed-forward layer.

    Methods:
        call: Performs the forward pass of the layer.

    Attributes:
        seq (tf.keras.Sequential): The sequential layer that contains the feed-forward layers. It applies the two feed-forward layers and the dropout layer.
        add (tf.keras.layers.Add): The Add layer.
        layer_norm (tf.keras.layers.LayerNormalization): The LayerNormalization layer.
    """
    def __init__(self, d_model: int, dff: int, dropout_rate: float=0.1):
        """
        Constructor of the FeedForward layer.

        Args:
            d_model (int): The dimensionality of the model.
            dff (int): The dimensionality of the feed-forward layer.
            dropout_rate (float): The dropout rate.
        """
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x: tf.Tensor) -> tf.Tensor:
        """
        The call function that performs the feed-forward operation. 

        Args:
            x (tf.Tensor): The input sequence of shape (batch_size, seq_length, d_model).

        Returns:
            tf.Tensor: The output sequence of shape (batch_size, seq_length, d_model).
        """
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [14]:
class EncoderLayer(tf.keras.layers.Layer):
    """
    A single layer of the Encoder. Usually there are multiple layers stacked on top of each other.

    Methods:
        call: Performs the forward pass of the layer.

    Attributes:
        self_attention (GlobalSelfAttention): The global self-attention layer.
        ffn (FeedForward): The feed-forward layer.
    """
    def __init__(self, d_model: int, num_heads: int, dff: int, dropout_rate: float=0.1):
        """
        Constructor of the EncoderLayer.

        Args:
            d_model (int): The dimensionality of the model.
            num_heads (int): The number of heads in the multi-head attention layer.
            dff (int): The dimensionality of the feed-forward layer.
            dropout_rate (float): The dropout rate.
        """
        super().__init__()

        self.self_attention = TimeSeriesSelfAttention(
            num_heads=num_heads,
            d_model=d_model,
            dropout_rate=dropout_rate
            )

        self.ffn = FeedForward(d_model, dff)

    def call(self, x: tf.Tensor) -> tf.Tensor:
        """
        The call function that performs the forward pass of the layer.

        Args:
            x (tf.Tensor): The input sequence of shape (batch_size, seq_length, d_model).

        Returns:
            tf.Tensor: The output sequence of shape (batch_size, seq_length, d_model).
        """
        x = self.self_attention(x)
        x = self.ffn(x)
        return x

In [15]:
class Encoder(Layer):
    """
    A custom TensorFlow layer that implements the Encoder.

    Methods:
        call: Performs the forward pass of the layer.

    Attributes:
        d_model (int): The dimensionality of the model.
        num_layers (int): The number of layers in the encoder.
        pos_encoding (Positionalncoding): The positional encoding layer.
        enc_layers (list): The list of encoder layers.
        dropout (tf.keras.layers.Dropout): The dropout layer.
    """
    def __init__(self, num_layers: int, d_model: int, num_heads: int, dff: int, max_length: int, dropout_rate: float=0.1):
        """
        Constructor of the Encoder.

        Args:
            num_layers (int): The number of layers in the encoder.
            d_model (int): The dimensionality of the model.
            num_heads (int): The number of heads in the multi-head attention layer.
            dff (int): The dimensionality of the feed-forward layer.
            max_length (int): The size of the max input sequence.
            dropout_rate (float): The dropout rate.
        """
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_encoding = PositionalEncoding(max_length=max_length, d_model=d_model)

        self.enc_layers = [
            EncoderLayer(d_model=d_model,
                        num_heads=num_heads,
                        dff=dff,
                        dropout_rate=dropout_rate)
            for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x: tf.Tensor) -> tf.Tensor:
        """
        The call function that performs the forward pass of the layer.
        
        Args:
            x (tf.Tensor): The input sequence of shape (batch_size, seq_length).

        Returns:
            tf.Tensor: The output sequence of shape (batch_size, seq_length, d_model).
        """
        x = self.pos_encoding(x)  
        # here x has shape `(batch_size, seq_len, d_model)`

        # Add dropout.
        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x)

        return x  # Shape `(batch_size, seq_len, d_model)`.

In [16]:
from tensorflow.keras.layers import Input, Dense, Concatenate, Flatten, Reshape, RepeatVector

def TimeSeriesTransformer(
    num_layers: int = 8, 
    d_model: int = 16, 
    num_heads: int = 2,
    dff: int = 16,
    window: int = window,
    telescope: int = telescope,
    num_categories: int = 6,
    dropout_rate: float = 0.2
) -> tf.keras.Model:
    """
    A custom TensorFlow model for time series forecasting using Transformer architecture with categorical inputs.
    """
   # Inputs
    time_series_input = Input(shape=(window, 1), dtype=tf.float32)  # Shape: (batch_size, window, 1)
    categorical_input = Input(shape=(num_categories,), dtype=tf.float32)  # Shape: (batch_size, num_categories)

    # Process Time Series Input
    ts_processed = Dense(d_model, activation='relu')(time_series_input)  # Shape: (batch_size, window, d_model)

    # Process Categorical Input
    cat_processed = Dense(d_model, activation='relu')(categorical_input)  # Shape: (batch_size, d_model)
    cat_processed = RepeatVector(window)(cat_processed)  # Shape: (batch_size, window, d_model)

    # Combine Time Series and Categorical Data
    combined_input = Concatenate(axis=-1)([ts_processed, cat_processed])  # Shape: (batch_size, window, 2*d_model)

    # Encoder
    encoder_output = Encoder(num_layers, 2*d_model, num_heads, dff, window, dropout_rate)(combined_input)  # Adjusted d_model to 2*d_model

    # Output Layer
    output = Dense(telescope)(encoder_output[:, -1, :])  # Predicting 'telescope' future values

    return tf.keras.Model(inputs=[time_series_input, categorical_input], outputs=output)

In [17]:
model = TimeSeriesTransformer()

2023-12-21 12:15:18.835590: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-21 12:15:18.874911: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-21 12:15:18.876571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [18]:
model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

In [19]:
model.summary()

from tensorflow import keras as tfk
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 input_1 (InputLayer)        [(None, 100, 1)]             0         []                            
                                                                                                  
 dense_1 (Dense)             (None, 16)                   112       ['input_2[0][0]']             
                                                                                                  
 dense (Dense)               (None, 100, 16)              32        ['input_1[0][0]']             
                                                                                              

In [20]:
batch_size = 64
epochs = 50

In [25]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    patience=20, 
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    mode='min', 
    patience=15, 
    factor=0.1, 
    min_lr=1e-5
)

history = model.fit(
    x = [X_train, X_train_category],
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = ([X_val, X_val_category], y_val),
    callbacks = [
        early_stopping,
        reduce_lr
    ]
).history

Epoch 1/50
13048/13048 [==============================] - 431s 33ms/step - loss: 0.0111 - val_loss: 0.0139 - lr: 1.0000e-04
Epoch 2/50
13048/13048 [==============================] - 432s 33ms/step - loss: 0.0110 - val_loss: 0.0131 - lr: 1.0000e-04
Epoch 3/50
13048/13048 [==============================] - 429s 33ms/step - loss: 0.0109 - val_loss: 0.0140 - lr: 1.0000e-04
Epoch 4/50
13048/13048 [==============================] - 429s 33ms/step - loss: 0.0109 - val_loss: 0.0132 - lr: 1.0000e-04
Epoch 5/50
13048/13048 [==============================] - 428s 33ms/step - loss: 0.0109 - val_loss: 0.0140 - lr: 1.0000e-04
Epoch 6/50
13048/13048 [==============================] - 429s 33ms/step - loss: 0.0108 - val_loss: 0.0136 - lr: 1.0000e-04
Epoch 7/50
13048/13048 [==============================] - 429s 33ms/step - loss: 0.0108 - val_loss: 0.0137 - lr: 1.0000e-04
Epoch 8/50
13048/13048 [==============================] - 430s 33ms/step - loss: 0.0108 - val_loss: 0.0136 - lr: 1.0000e-04
Epoch 9/

In [27]:
model.save('transformer_med')

INFO:tensorflow:Assets written to: transformer_med/assets


INFO:tensorflow:Assets written to: transformer_med/assets


In [28]:
!zip -r transformer_med.zip ./transformer_med

  adding: transformer_med/ (stored 0%)
  adding: transformer_med/keras_metadata.pb (deflated 95%)
  adding: transformer_med/saved_model.pb (deflated 92%)
  adding: transformer_med/variables/ (stored 0%)
  adding: transformer_med/variables/variables.index (deflated 78%)
  adding: transformer_med/variables/variables.data-00000-of-00001 (deflated 18%)
  adding: transformer_med/fingerprint.pb (stored 0%)
  adding: transformer_med/assets/ (stored 0%)
